In [25]:
import pandas as pd
import requests

FY2011_DRG_INFO = 'https://data.cms.gov/resource/ehrv-m9r6.json?'
FY2012_DRG_INFO = 'https://data.cms.gov/resource/d5xj-qqyf.json?'
FY2013_DRG_INFO = 'https://data.cms.gov/resource/dxb9-6mzu.json?'
FY2014_DRG_INFO = 'https://data.cms.gov/resource/5xma-hdru.json?'
FY2015_DRG_INFO = 'https://data.cms.gov/resource/spmh-qjdv.json?'

resp_2011 = requests.get(FY2011_DRG_INFO)
resp_2012 = requests.get(FY2012_DRG_INFO)

# Confirm request was valid:
print(resp_2011)

#Return a dictionary containing JSON parsed into Python objects:
data_2011 = resp_2011.json()
data_2012 = resp_2012.json()

# Column headers: First row of data
print(data_2011[0])
print(data_2012[0])

# Generate a DF
providers_drg_FY2011 = pd.DataFrame(data_2011, columns=data_2011[0])
providers_drg_FY2012 = pd.DataFrame(data_2012, columns=data_2012[0])
#print(providers_drg)

providers_drg_FY2011.describe()
providers_drg_FY2012.describe()

<Response [200]>
{u'provider_id': u'10001', u'average_medicare_payments_2': u'4763.73', u'provider_city': u'DOTHAN', u'average_medicare_payments': u'5777.24', u'provider_zip_code': u'36301', u'average_covered_charges': u'32963.07', u'provider_street_address': u'1108 ROSS CLARK CIRCLE', u'hospital_referral_region_description': u'AL - Dothan', u'provider_state': u'AL', u'provider_name': u'SOUTHEAST ALABAMA MEDICAL CENTER', u'drg_definition': u'039 - EXTRACRANIAL PROCEDURES W/O CC/MCC', u'total_discharges': u'91'}
{u'provider_id': u'50128', u'provider_city': u'OCEANSIDE', u'average_medicare_payments': u'4167.55', u'provider_zip_code': u'92056', u'average_covered_charges': u'24412.2', u'provider_street_address': u'4002 VISTA WAY', u'average_total_payments': u'5519.2', u'provider_state': u'CA', u'provider_name': u'TRI-CITY MEDICAL CENTER', u'hospital_referral_region_hrr_description': u'CA - San Diego', u'drg_definition': u'069 - TRANSIENT ISCHEMIA', u'total_discharges': u'20'}


,provider_id,provider_city,average_medicare_payments,provider_zip_code,average_covered_charges,provider_street_address,average_total_payments,provider_state,provider_name,hospital_referral_region_hrr_description,drg_definition,total_discharges
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,915,644,999,883,1000,914,998,48,894,269,51,102
top,390063,BALTIMORE,5323,16550,73601.94444,201 STATE STREET,6285.5,FL,UPMC HAMOT,MA - Boston,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,16
freq,7,10,2,7,1,7,2,90,7,25,821,53


In [23]:
# Data Exploration:

#num_providers = Counter(providers_drg.provider_id)

# print(num_providers)

# Range: Convert Object to Int Series
disch_2011_Series = pd.Series(providers_drg_FY2011['total_discharges'].astype(str).astype(int))

print(disch_2011_Series.describe())

print("Min Number of Discharges 2011: " + str(disch_2011_Series.min(axis=0)))
print("Max Number of Discharges 2011: " + str(disch_2011_Series.max(axis=0)))
# print("Max Number of Discharges: " + str(max_num_disch))

disch_2012_Series = pd.Series(providers_drg_FY2012['total_discharges'].astype(str).astype(int))

print(disch_2012_Series.describe())

print("Min Number of Discharges 2012: " + str(disch_2012_Series.min(axis=0)))
print("Max Number of Discharges 2012: " + str(disch_2012_Series.max(axis=0)))
# print("Max Number of Discharges: " + str(max_num_disch))

count    1000.000000
mean       31.403000
std        21.852234
min        11.000000
25%        16.000000
50%        24.000000
75%        40.000000
max       190.000000
Name: total_discharges, dtype: float64
Min Number of Discharges 2011: 11
Max Number of Discharges 2011: 190
count    1000.000000
mean       32.842000
std        27.314046
min        11.000000
25%        16.000000
50%        25.000000
75%        40.000000
max       285.000000
Name: total_discharges, dtype: float64
Min Number of Discharges 2012: 11
Max Number of Discharges 2012: 285


In [28]:
# MERGING DATA SETS:

fy2011_subset = providers_drg_FY2011[:][0:5]
fy2012_subset = providers_drg_FY2012[:][0:5]

fy2011_2012_subset = pd.concat([fy2011_subset, fy2012_subset], keys = ['2011', '2012'])

print(fy2011_2012_subset)

       average_covered_charges average_medicare_payments  \
2011 0                32963.07                   5777.24   
     1                15131.85                   5787.57   
     2                37560.37                   5434.95   
     3                13998.28                   5417.56   
     4                31633.27                   5658.33   
2012 0                 24412.2                   4167.55   
     1              24080.1875                 3213.3125   
     2             11446.15385               4143.538462   
     3                11985.19                   3545.97   
     4             10849.45833                    2754.5   

       average_medicare_payments_2 average_total_payments  \
2011 0                     4763.73                    NaN   
     1                     4976.71                    NaN   
     2                     4453.79                    NaN   
     3                     4129.16                    NaN   
     4                     4851.44

In [24]:
# Creating buckets based on average_medicare_payment
def buckets(data_point, bucket_size):
    return bucket_size * math.floor(data_point / bucket_size)

# buckets(providers_drg.total_discharges, )